# Section 1: Load Model

In [1]:
# For dataset
import pandas as pd
import json
import os
import ast
import re
import numpy as np
from datasets import Dataset
import matplotlib.pyplot as plt

# For LLM
from peft import LoraConfig, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    set_seed,
    pipeline
)
from trl import SFTTrainer, setup_chat_format, SFTConfig, DataCollatorForCompletionOnlyLM

import torch
from time import time
 
# For wandb
import wandb
# Set seed
import pickle
set_seed(42)
from transformers.tokenization_utils_fast import PreTrainedTokenizerFast
from transformers import BatchEncoding, DataCollatorForSeq2Seq
from unsloth import FastLanguageModel
from peft import LoraConfig, PeftModel, get_peft_model, get_peft_model_state_dict
import seaborn as sns
from copy import deepcopy
from tqdm import tqdm
PATH = "0_Thesis/4_Multi/"

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
if os.getcwd() == '/root':
    new_path = "/root/0_Thesis/0_final/"
    os.chdir(new_path)
else:
    os.chdir("..") 
print(os.getcwd())

/root/0_Thesis/0_final


In [ ]:
model_id = "unsloth/Llama-3.2-1B-Instruct-bnb-4bit"
# model_id ="unsloth/Qwen2.5-14B-Instruct-bnb-4bit"
# model_id = "unsloth/gemma-2-9b-it-bnb-4bit"

In [9]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_id,
    max_seq_length = 500,
    dtype = getattr(torch, "float16"),
    attn_implementation='eager'
)


==((====))==  Unsloth 2025.2.5: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: NVIDIA RTX A6000. Max memory: 47.529 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Device supports bfloat16 but you selected float16. Will change to bfloat16.


# Section 2: Load Data

In [15]:
df = pd.read_csv("data/binary/m2.v2.csv")

In [17]:
df.shape

(240647, 12)

In [18]:
df.columns

Index(['index', 'text', 'prompt', 'token_len',
       'unsloth/Qwen2.5-14B-Instruct-bnb-4bit_label_1',
       'unsloth/Qwen2.5-14B-Instruct-bnb-4bit_label_2',
       'unsloth/gemma-2-9b-it-bnb-4bit_label_1',
       'unsloth/gemma-2-9b-it-bnb-4bit_label_2',
       'unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit_label_1',
       'unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit_label_2',
       'unsloth/mistral-7b-instruct-v0.3-bnb-4bit_label_1',
       'unsloth/mistral-7b-instruct-v0.3-bnb-4bit_label_2'],
      dtype='object')

# Section 3: Labeling

In [ ]:
mstral7b = "unsloth/mistral-7b-instruct-v0.3-bnb-4bit"
llama8b = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"
gemma9b = "unsloth/gemma-2-9b-it-bnb-4bit"
qwen14b = "unsloth/Qwen2.5-14B-Instruct-bnb-4bit"

Individual Model

In [38]:
#Gemma 9B-0.5
df["mean_label_1"] = df[gemma9b+ "_label_1"]
df["mean_label_2"] = df[gemma9b+ "_label_2"]

index_1 = df['mean_label_1'] > df['mean_label_2']
index_2 = df['mean_label_1'] > .5 # Threshold selection
index_all = index_1 & index_2
df['label_id'] = 2
df.loc[index_all, 'label_id'] = 1
print("Percent Hate Labeled: ", round(np.sum(index_all)/df.shape[0] *100,2))
df['label_id'].value_counts()

Percent Hate Labeled:  6.03


label_id
2    226141
1     14506
Name: count, dtype: int64

Mean Model

In [28]:
lb_1 = df[qwen14b+ "_label_1"] * .25  + df[llama8b + "_label_1"] * .25 + df[gemma9b+ "_label_1"] * .25 + df[mstral7b+ "_label_1"] * .25
lb_2 = df[qwen14b+ "_label_2"] * .25 + df[llama8b + "_label_2"] * .25 + df[gemma9b+ "_label_2"] * .25 + df[mstral7b+ "_label_2"] * .25
 
df["mean_label_1"] = lb_1
df["mean_label_2"] = lb_2

index_1 = df['mean_label_1'] > df['mean_label_2']
index_2 = df['mean_label_1'] > .1 # Threshold selection
index_all = index_1 & index_2
df['label_id'] = 2
df.loc[index_all, 'label_id'] = 1
print("Percent Hate Labeled: ", round(np.sum(index_all)/df.shape[0] *100,2))
df['label_id'].value_counts()

Percent Hate Labeled:  1.66


label_id
2    236660
1      3987
Name: count, dtype: int64

Vote 2/3 Model

In [37]:
mstral7b = "unsloth/mistral-7b-instruct-v0.3-bnb-4bit"
llama8b = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"
gemma9b = "unsloth/gemma-2-9b-it-bnb-4bit"
qwen14b = "unsloth/Qwen2.5-14B-Instruct-bnb-4bit"

index_1 = df[qwen14b+ "_label_1"] > df[qwen14b+ "_label_2"]
index_2 = df[qwen14b+ "_label_1"] > .1
index_qwen = index_1 & index_2

index_1 = df[llama8b+ "_label_1"] > df[llama8b+ "_label_2"]
index_2 = df[llama8b+ "_label_1"] > .1
index_llama8b = index_1 & index_2


index_1 = df[gemma9b+ "_label_1"] > df[gemma9b+ "_label_2"]
index_2 = df[gemma9b+ "_label_1"] > .1
index_gemma9b = index_1 & index_2

index_1 = df[mstral7b+ "_label_1"] > df[mstral7b+ "_label_2"]
index_2 = df[mstral7b+ "_label_1"] > .1
index_mstral7b = index_1 & index_2

index_all = index_mstral7b.astype(int) + index_gemma9b.astype(int) + index_llama8b.astype(int) + index_qwen.astype(int)

index_all = index_all >= 2 #Vote2
# index_all = index_all >= 3 #Vote3 


df['label_id'] = 2

df.loc[index_all, 'label_id'] = 1
print("Percent Hate Labeled: ", round(np.sum(index_all)/df.shape[0] *100,2))
df['label_id'].value_counts()


Percent Hate Labeled:  2.03


label_id
2    235754
1      4893
Name: count, dtype: int64

LGB

In [43]:
c1 = ['unsloth/Qwen2.5-14B-Instruct-bnb-4bit_label_1', 'unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit_label_1', 'unsloth/mistral-7b-instruct-v0.3-bnb-4bit_label_1', 'unsloth/gemma-2-9b-it-bnb-4bit_label_1', 
        ]
with open("lgb/lgb_binary_label_1.pkl", "rb") as f:
    lgb_model = pickle.load(f)
X = df[c1]
val_preds = lgb_model.predict(X, num_iteration=lgb_model.best_iteration)
df["mean_label_1"] = val_preds

with open("lgb/lgb_binary_label_2.pkl", "rb")  as f:
    lgb_model = pickle.load(f)
X =df[['unsloth/Qwen2.5-14B-Instruct-bnb-4bit_label_2', 'unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit_label_2', 'unsloth/mistral-7b-instruct-v0.3-bnb-4bit_label_2', 'unsloth/gemma-2-9b-it-bnb-4bit_label_2', 
       ]]
val_preds = lgb_model.predict(X, num_iteration=lgb_model.best_iteration)
df["mean_label_2"] = val_preds

index_1 = df['mean_label_1'] > df['mean_label_2']
index_2 = df['mean_label_1'] > .1
index_all = index_1 & index_2
df['label_id'] = 2
df.loc[index_all, 'label_id'] = 1
index_lgb = deepcopy(index_all)

print("Percent Hate Labeled: ", round(np.sum(index_all)/df.shape[0] *100,2))
df['label_id'].value_counts()


Percent Hate Labeled:  1.15


label_id
2    237883
1      2764
Name: count, dtype: int64

In [44]:
user_message_template = '''You are tasked with annotating speech. Your response must be a single valid number:
1 for Hate Speech,
2 for Normal Speech.

Provide only the number corresponding to the category. Do not include any explanation or additional text.
Do you think the following comment is hate speech or offensive speech?
\n"{comment}"\n
Your Answer:
'''

def preprocess(text, label):

    user_message_content = user_message_template.format(comment=text)
    user_message = {
        "role": "user",
        "content": user_message_content
    }
    assistant_message = {
        "role": "assistant",
        "content": str(label)
    }
    
    if "Qwen" in model_id:
        system_message =  {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant"}
    else:
        system_message =  {"role": "system", "content": "You are a helpful assistant"}
    if "gemma" in model_id or "gemma" in model_id:
        messages = [user_message, assistant_message]
        messages = tokenizer.apply_chat_template(messages, tokenize=False)

    else:
        messages = [system_message, user_message, assistant_message]
        messages = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    messages = messages
    return messages

df["prompt"] = df.apply(lambda row: preprocess(row["text"], row["label_id"]), axis=1)
comments = df["prompt"].tolist()
dataset = Dataset.from_pandas(df)
filtered_dataset = dataset.shuffle(seed=42)


# Section 4: Lora

In [ ]:
output_dir = PATH + "m2.v2.lgb.1B"
sft_config = SFTConfig(
    output_dir=output_dir, 
    eval_strategy="no", 
    do_eval=False,  
    optim="paged_adamw_8bit", 
    per_device_train_batch_size=4, 
    gradient_accumulation_steps=4,  
    per_device_eval_batch_size=1,  
    log_level="debug",  
    save_steps=1000,
    logging_steps=200, 
    learning_rate=1e-5,  

    eval_steps=5000,  
    max_steps=50, 
    # max_steps=-1, 

    num_train_epochs=1, 
    warmup_steps=1, 
    lr_scheduler_type="cosine", 
    fp16=False,  
    bf16=True, 
    max_grad_norm=0.2, 
    gradient_checkpointing=True, 
    gradient_checkpointing_kwargs={'use_reentrant':False},

    dataset_text_field="prompt",
    max_seq_length=400, 
    packing=False,
)

In [12]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 8, 
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, 
    bias = "none", 
    use_gradient_checkpointing = "unsloth", 
    random_state = 3407,
    use_rslora = True, 
    loftq_config = None, 
)

Unsloth 2025.2.5 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


In [14]:
model.gradient_checkpointing_enable()
tokenizer.pad_token = tokenizer.eos_token 
trainer = SFTTrainer(
        model=model,
        train_dataset=filtered_dataset,  
        tokenizer=tokenizer, 
        args=sft_config, 
)

trainer.model.print_trainable_parameters()
trainer.train()


Map:   0%|          | 0/343709 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
Using auto half precision backend


trainable params: 5,636,096 || all params: 1,241,450,496 || trainable%: 0.4540


Currently training with a batch size of: 8
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 343,709 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 8 | Gradient Accumulation steps = 8
\        /    Total batch size = 64 | Total steps = 50
 "-____-"     Number of trainable parameters = 5,636,096
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: winter2109. Use `wandb login --relogin` to force relogin


Step,Training Loss


Saving model checkpoint to 0_Thesis/4_Multi/multi.5.lgb.llama1B/checkpoint-50
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--unsloth--llama-3.2-1b-instruct-unsloth-bnb-4bit/snapshots/0a4436e20494a6504464ce35274b7e53fb7883d0/config.json
Model config LlamaConfig {
  "_name_or_path": "meta-llama/Llama-3.2-1B-Instruct",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128009,
  "head_dim": 64,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 16,
  "num_key_value_heads": 8,
  "pad_token_id": 128004,
  "pretraining_tp": 1,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_

TrainOutput(global_step=50, training_loss=2.4246192932128907, metrics={'train_runtime': 187.8131, 'train_samples_per_second': 17.038, 'train_steps_per_second': 0.266, 'total_flos': 6677722038435840.0, 'train_loss': 2.4246192932128907, 'epoch': 0.009310120100549298})